In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd

PATH = ("/content/drive/MyDrive/Spotify_Project/4_Feature_Engineering/spotify_with_features.pkl")

with open(PATH, "rb") as f:
    DF = pickle.load(f)

print("Loaded:", DF.shape)
display(DF.head())

Loaded: (32833, 57)


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,has_with,has_dont,has_remastered,has_version,has_like,has_your,has_original,has_remaster,has_down,has_this
0,6f807x0ima9a1j3VPbc7VN,i dont care with justin bieber loud luxury remix,ed sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,i dont care with justin bieber loud luxury remix,2019-06-14,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,1,0,0,0,0,0,0,0,0
1,0r7CVbZTWZgbTCYdfa2P31,memories dillon francis remix,maroon 5,67,63rPSO264uRjW1X5E6cWv6,memories dillon francis remix,2019-12-13,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
2,1z1Hg7Vb0AhHDiEmnDE79l,all the time don diablo remix,zara larsson,70,1HoSmj2eLcsrR0vE9gThr4,all the time don diablo remix,2019-07-05,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
3,75FpbthrwQmzHlBJLuGdC7,call you mine keanu silva remix,the chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,call you mine the remixes,2019-07-19,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
4,1e8PAfcKUYoKkxPhrHqw4x,someone you loved future humans remix,lewis capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,someone you loved future humans remix,2019-03-05,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
drop_cols = [
    "track_id", "playlist_id", "track_album_id",
    "track_name", "track_artist",
    "track_album_name", "playlist_name",
    "track_album_release_date"
]

DF_model = DF.drop(columns=drop_cols, errors="ignore")
print("Remaining columns:", DF_model.shape[1])

Remaining columns: 49


In [ ]:
TARGET = "track_popularity"

y = DF_model[TARGET]
X = DF_model.drop(columns=[TARGET])
print("X shape:", X.shape, " | y shape:", y.shape)

X shape: (32833, 48)  | y shape: (32833,)


Encoding

In [ ]:
X_enc = pd.get_dummies(X, drop_first=True)
print("After encoding:", X_enc.shape)

After encoding: (32833, 94)


#Feature Selection

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

lasso = Lasso(alpha=5, max_iter=10000, random_state=42).fit(X_enc, y)
ridge = Ridge(alpha=5, random_state=42).fit(X_enc, y)
gb = GradientBoostingRegressor(random_state=42).fit(X_enc, y)
rf = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1).fit(X_enc, y)

In [ ]:
import numpy as np
import pandas as pd

lasso_sel = (np.abs(lasso.coef_) > 0).astype(int)
ridge_sel = (np.abs(ridge.coef_) > 0.05 * np.abs(ridge.coef_).max()).astype(int)
gb_sel = (gb.feature_importances_ > gb.feature_importances_.mean()).astype(int)
rf_sel = (rf.feature_importances_ > rf.feature_importances_.mean()).astype(int)

selection_df = pd.DataFrame({
    "Feature": X_enc.columns,
    "Lasso": lasso_sel,
    "Ridge": ridge_sel,
    "GradientBoost": gb_sel,
    "RandomForest": rf_sel
})

selection_df["Sum"] = selection_df[["Lasso","Ridge","GradientBoost","RandomForest"]].sum(axis=1)
selection_df = selection_df.sort_values("Sum", ascending=False)
print(selection_df.head(20))

                                        Feature  Lasso  Ridge  GradientBoost  \
19                          song_playlist_count      1      1              1   
1                                        energy      0      1              1   
16                           artist_album_count      1      0              1   
20                          playlist_song_count      1      0              1   
17                        artist_playlist_count      1      0              1   
12                         instrumentalness_log      0      1              1   
22                         playlist_album_count      1      0              1   
21                        playlist_artist_count      1      0              1   
63  playlist_subgenre_progressive electro house      0      1              1   
60             playlist_subgenre_permanent wave      0      1              1   
2                                      loudness      0      1              0   
3                                   spee

In [ ]:
selection_df["Sum"].value_counts().sort_index()

,count
Sum,
0,22
1,47
2,15
3,9
4,1


Selection Criteria
Features with ≥2 model votes (out of 4: Lasso, Ridge, Random Forest, Gradient Boosting) were selected for inclusion.
10 strong predictors identified by 3-4 models (high model agreement).
15 additional predictors identified by 2 models, showing consistent but moderate importance.
Total: 25 selected features retained for the modeling phase.

In [ ]:
selected_features = selection_df.loc[selection_df["Sum"] >= 2, "Feature"]
print("Number of selected features:", len(selected_features))

Number of selected features: 25


In [ ]:
print(selected_features)

19                            song_playlist_count
1                                          energy
16                             artist_album_count
20                            playlist_song_count
17                          artist_playlist_count
12                           instrumentalness_log
22                           playlist_album_count
21                          playlist_artist_count
63    playlist_subgenre_progressive electro house
60               playlist_subgenre_permanent wave
2                                        loudness
3                                     speechiness
9                                    release_year
6                                         valence
7                                           tempo
8                                     duration_ms
10                                  release_month
11                                    release_day
25                       playlist_name_word_count
18                               album_song_count


In [ ]:
!ls "/content/drive/MyDrive/Spotify_Project/"

1_Prep					  spotify_for_model.pkl
2_EDA					  spotify_max.xlsx
3_Cleansing				  spotify_min.xlsx
4_Feature_Engineering			  spotify_missing.xlsx
5_Data_Selection			  spotify_model_df_17feats.pkl
6_Models_Finetuning			  spotify_raw.parquet
outlier_summary.xlsx			  spotify_raw.pkl
spotify_clean_ready_2025-10-25_08-27.pkl  spotify_sweetviz.html
spotify_datatypes.xlsx			  spotify_unique.xlsx
spotify_EDA_full.pkl			  spotify_with_features.pkl
spotify_EDA.html


In [ ]:
# Create final DataFrame with only selected features + target
X_selected = X_enc[selected_features].copy()
DF_selected = pd.concat([X_selected, y], axis=1)

# Save the real dataset (not the votes table)
import pickle
with open("/content/drive/MyDrive/Spotify_Project/5_Data_Selection/spotify_selected_features.pkl", "wb") as f:
    pickle.dump(DF_selected, f)

print("Saved: DF_selected with data ", DF_selected.shape)

Saved: DF_selected with data  (32833, 26)
